<a href="https://colab.research.google.com/github/karino2/tegashiki/blob/master/tegashiki_symbol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tegashiki one symbol training

Training for feature extractor

In [0]:
import os
import datetime
import pickle
import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing


In [0]:
from google.colab import auth
auth.authenticate_user()

W0709 10:03:13.240982 140250003937152 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
BEGIN_OF_SEQ = 113
END_OF_SEQ=0
VOCAB_SIZE=114


NORMALIZE_MAX=2000


MAX_ONE_STROKE_LEN=50
MAX_STROKE_NUM=10


INPUT_TYPE_POINT=1
INPUT_TYPE_END=0

# (x, y, TYPE)
INPUT_TYPE_DIM=3

In [0]:
tf.VERSION

'1.14.0'

### Model

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, RepeatVector, Reshape, Concatenate
from tensorflow.keras.layers import TimeDistributed, Flatten, Lambda, Add, Activation, Masking, Embedding
from tensorflow.keras.layers import AveragePooling1D, Conv1D, MaxPooling1D, SpatialDropout1D
from tensorflow.keras.layers import Dropout, GlobalMaxPooling1D
from tensorflow.keras import regularizers
import  tensorflow.keras.layers as layers
# from tensorflow.keras.layers.embeddings import Embedding
import tensorflow.keras.backend as K

In [0]:
DROPOUT_RATE=0.5
L2_REGULARIZATION_RATE=0.1

# DROPOUT_RATE=0.9
#L2_REGULARIZATION_RATE=0.1

# EXTRACTED_FETURE_DIM=512
EXTRACTED_FETURE_DIM=256
FEATURE_EXTRACTER_KERNEL_SIZE=7



# large, 
GRU_HIDDEN=256

# model_small
# GRU_HIDDEN=128



 https://arxiv.org/abs/1803.01271

try to cover 500 len. and attention will handle larger case.
As paper noted, best kernel size depend on task.
I start from k=8 because our task is somewhat similar to P-MNIST, and k=8 is best for that task.

In [0]:
# TCN residual block in paper
# filter_size must be the same as out_channels?

# filter_size=32
# filter_size=25
def TCNResBlock(input, layer_depth, filter_size=10, kernel_size=8, dropout_rate=0.2):
  d = 2**layer_depth
  def conv(x):
    if L2_REGULARIZATION_RATE == 0.0:
      return Conv1D(filter_size, kernel_size, activation='relu', dilation_rate=d, padding='causal')(x)
    else:
      return Conv1D(filter_size, kernel_size, activation='relu', dilation_rate=d, padding='causal', kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(x)
    
  x = conv(input)
  
  # https://github.com/ychfan/tf_estimator_barebone/blob/master/common/layers.py
  # weight norm implementation. But I use layer_norm for first trial.
  x = tf.contrib.layers.layer_norm(x)
  x = SpatialDropout1D(dropout_rate)(x)

  x = conv(x)
  x = tf.contrib.layers.layer_norm(x)
  x = SpatialDropout1D(dropout_rate)(x)
  
  res_input = input
  if filter_size != 3:
    # 1 by 1 conv to match inupt dim.
    res_input = Conv1D(filter_size, 1, padding='same')(input)
  res_input = res_input
  return tf.nn.relu(x + res_input)

def TCN(input, depth=8):
  x = input
  for i in range(depth):
    x = TCNResBlock(x, i, dropout_rate=DROPOUT_RATE)
  return x


In [0]:
def crnn_base(input_stroke_t):
  conved = Conv1D(32, 7, activation='relu', kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(input_stroke_t)
  last = GRU(GRU_HIDDEN, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), recurrent_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(conved)
  return last

def tcn_base(input_stroke_t):
  tcn = TCN(input_stroke_t, 5)
  last = tcn[:, -1, :]
  return last

In [0]:
# input_stroke_t = tf.placeholder(tf.float32, shape=[None, MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM])
# x = create_model(input_stroke_t)
# x = feature_extractor(input_stroke_t)
# x.shape

In [0]:
def fe_conv1d(filternum, kernelsize, x):
    return Conv1D(filternum, kernelsize, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(x)
  

def feature_extractor(input_stroke_t, is_training):
  """input_stroke_t shape (batch, MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM)"""
  with tf.variable_scope("feature_extractor"):
    inpshape = input_stroke_t.shape
    x = tf.reshape(input_stroke_t, [-1, inpshape[2], inpshape[3]])
    # (batch*MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM)

    x = fe_conv1d(32, FEATURE_EXTRACTER_KERNEL_SIZE, x)
    x = tf.layers.BatchNormalization()(x, training=is_training)
    x = Activation('relu')(x)
    # (batch*MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, 32)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(DROPOUT_RATE)(x, training=is_training)
    # (batch*MAX_STROKE_NUM, MAX_ONE_STROKE_LEN/2, 32)

    x = fe_conv1d(64, FEATURE_EXTRACTER_KERNEL_SIZE, x)
    x = tf.layers.BatchNormalization()(x, training=is_training)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(DROPOUT_RATE)(x, training=is_training)
    # (batch*MAX_STROKE_NUM, MAX_ONE_STROKE_LEN/4, 64)

    x = fe_conv1d(EXTRACTED_FETURE_DIM, FEATURE_EXTRACTER_KERNEL_SIZE, x)
    x = tf.layers.BatchNormalization()(x, training=is_training)
    x = Activation('relu')(x)
    x = Dropout(DROPOUT_RATE)(x, training=is_training)
    x = GlobalMaxPooling1D()(x)
    x = tf.reshape(x, [-1, inpshape[1], EXTRACTED_FETURE_DIM])
    return x
  
  

    

In [0]:
def create_model(input_stroke_t, is_training):
  features = feature_extractor(input_stroke_t, is_training)
  # (batch, MAX_STROKE_NUM, EXTRACTED_FETURE_DIM)
  last = GRU(GRU_HIDDEN, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), recurrent_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(features, training=is_training)

  
  # last = crnn_base(input_stroke_t)
  # last = tcn_base(input_stroke_t)

  if L2_REGULARIZATION_RATE == 0.0:
    logit = Dense(VOCAB_SIZE)(last)
  else:
    logit = Dense(VOCAB_SIZE, kernel_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), bias_regularizer=regularizers.l2(L2_REGULARIZATION_RATE), activity_regularizer=regularizers.l2(L2_REGULARIZATION_RATE))(last)
 
  return logit


## TPUEstimator

In [0]:
from tqdm.autonotebook import tqdm as tqdmn

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [0]:
DATA_DIR="gs://karino2-tegashiki/dataset"
TF_RECORD_FILE="{}/crohme2019_sym_50arr.tfrecord.gz".format(DATA_DIR)
TF_VALID_RECORD_FILE="{}/crohme2019_sym_50arr_valid.tfrecord.gz".format(DATA_DIR)

In [0]:
# MODEL_DIR="gs://karino2-tegashiki/sym_models/rnn_small_dropout05"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/tcn_large_nonreg"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/tcn_large_nonreg_filter25"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/tcn_large_nonreg_filter25_fix"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/tcn_large_dropout05"

# MODEL_DIR="gs://karino2-tegashiki/sym_models/rnn_large_nonreg"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/tcn_filter25_dropout09"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/tcn_filter25_dropout09_datanormalize"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/tcn_pad_valid"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/tcn_filter10"

# MODEL_DIR="gs://karino2-tegashiki/sym_models/rnn_dropout05_pad"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/rnn_fdim256"

# MODEL_DIR="gs://karino2-tegashiki/sym_models/fe_trainifix_v1_14"
# MODEL_DIR="gs://karino2-tegashiki/sym_models/fe_trainifix_batch8x256"

# MODEL_DIR="gs://karino2-tegashiki/sym_models/fe_movingavg_fix"
MODEL_DIR="gs://karino2-tegashiki/sym_models/fe_layersbatchnorm"

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.127.205.194:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 16877991905234623012),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2386136422821470208),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6085890531861965010),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8176900991778660416),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 7088486485206163817),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 13850533938513974826),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 14322764921635201157),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 11172012000168160484),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1134341668

In [0]:
VALID_SAMPLE_NUM=10019 # len(valid_sym_labels)
TRAIN_STEP_PER_ONCE=1000
EVAL_BATCH_SIZE=8

In [0]:
def parser(serialized_example):
  featurelen = MAX_STROKE_NUM*MAX_ONE_STROKE_LEN
  features = tf.parse_single_example(
      serialized_example,
      features={
      'input_x': tf.FixedLenFeature([featurelen], tf.int64),
      'input_y': tf.FixedLenFeature([featurelen], tf.int64),
      'input_type': tf.FixedLenFeature([featurelen], tf.int64),
      'label':tf.FixedLenFeature([], tf.int64)
      })

  input_x, input_y, input_type = [tf.reshape(tf.cast(features[fname], tf.int32), [MAX_STROKE_NUM, MAX_ONE_STROKE_LEN]) for fname in ['input_x', 'input_y', 'input_type']]
  # input_x = tf.reshape(tf.cast(features['input_x'], tf.int32), [MAX_STROKE_NUM, MAX_ONE_STROKE_LEN])
  # input_y = tf.reshape(tf.cast(features['input_y'], tf.int32)
  # input_type = tf.cast(features['input_type'], tf.int32)
  one_sample_stroke = tf.stack([input_x, input_y, input_type], 2)
  labels = tf.cast(features["label"], tf.int32)
  
  return {"stroke": one_sample_stroke}, labels



In [0]:

def tpu_input_fn(params):
  dataset = tf.data.TFRecordDataset(TF_RECORD_FILE, "GZIP")
  dataset = dataset.map(parser)
  dataset = dataset.shuffle(1000).repeat()
  dataset = dataset.batch(params['batch_size'], drop_remainder=True)
  return dataset

def tpu_input_fn_valid(params):
  dataset = tf.data.TFRecordDataset(TF_VALID_RECORD_FILE, "GZIP")
  dataset = dataset.map(parser)
  # dataset = dataset.padded_batch(params['batch_size'],({'stroke': [MAX_STROKE_SEQ_LEN, INPUT_TYPE_DIM]}, []), drop_remainder=True)
  dataset = dataset.batch(params['batch_size'], drop_remainder=True)
  return dataset



In [0]:

def metric_fn(labels, predicted_classes):
    """Function to return metrics for evaluation."""
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name="acc_op")
      
    return {"accuracy": accuracy}

def extract_params(features, mode, params):  
  input_stroke = tf.feature_column.input_layer(features, params['input_stroke'])

  #[MAX_STROKE_NUM, MAX_ONE_STROKE_LEN]
  input_stroke = tf.reshape(input_stroke, shape=(-1,MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM))
  return input_stroke

def extract_params_always_train(features, mode, params):
  return extract_params(features, tf.estimator.ModeKeys.TRAIN, params)
  
def tpu_model_fn(features, labels, mode, params):
  input_stroke = extract_params(features, mode, params)
  # input_stroke, input_for_dec, maxtklen = extract_params_always_train(features, mode, params)
  
  logit = create_model(input_stroke, mode==tf.estimator.ModeKeys.TRAIN)
  
  predicted_classes = tf.math.argmax(logit,axis=1)
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {
        "class_ids": predicted_classes[:, tf.newaxis],
        "logits": logit,
    }
    return tf.contrib.tpu.TPUEstimatorSpec(mode, predictions=predictions)  
  
  loss = tf.losses.sparse_softmax_cross_entropy(labels, logit)

  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, loss=loss, eval_metrics=(metric_fn, [labels, predicted_classes]))
  
  optimizer = tf.train.AdamOptimizer(params['learning_rate'])  
  optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)  
  
  train_op = tf.contrib.training.create_train_op(loss, optimizer)
  
  # train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
  return tf.contrib.tpu.TPUEstimatorSpec(mode, loss=loss, train_op=train_op)


In [0]:
def get_global_step(estimator):
  try:
    return int(estimator.get_variable_value("global_step"))
  except ValueError:
    return 0
      
def train_tpu_estimator(tpu_estimator, max_steps):
  step = get_global_step(tpu_estimator)+TRAIN_STEP_PER_ONCE
  while step < max_steps:
    tpu_estimator.train(
      input_fn = tpu_input_fn,
      max_steps=step)
    eval_results = tpu_estimator.evaluate(
      input_fn=tpu_input_fn_valid,
      steps= VALID_SAMPLE_NUM// EVAL_BATCH_SIZE)
    print(step)
    print(eval_results)
    step += TRAIN_STEP_PER_ONCE
  tpu_estimator.train(
    input_fn = tpu_input_fn,
    max_steps=max_steps)
  eval_results = tpu_estimator.evaluate(
    input_fn=tpu_input_fn_valid,
    steps= VALID_SAMPLE_NUM// EVAL_BATCH_SIZE)


In [0]:
cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2


### TPUEstimator instantiation

In [0]:
run_config = tf.contrib.tpu.RunConfig(
    cluster=cluster_resolver,
    master=None,
    model_dir=MODEL_DIR,
    save_checkpoints_steps=100,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=1000,
        num_shards=8,
        per_host_input_for_training=is_per_host
        # per_host_input_for_training=False
    ))

tpu_estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=tpu_model_fn,
    config=run_config,
    export_to_tpu=False, # Conv1D cause error for TPU graph with ReadVariableOp. why?
    params={
        'learning_rate': 0.00009,
#        'learning_rate': 0.001,
        'input_stroke':tf.feature_column.numeric_column(key="stroke", shape=(MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM)),
        # 'input_stroke_valid':tf.feature_column.numeric_column(key="stroke", shape=(MAX_VALID_STROKE_LEN, INPUT_TYPE_DIM)),
    },
    train_batch_size=8*256,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size=8)

### TPUEstimator train

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [28]:
train_tpu_estimator(tpu_estimator, 200000)

115000
{'accuracy': 0.85682905, 'loss': 0.57964367, 'global_step': 115000}
116000
{'accuracy': 0.85603034, 'loss': 0.58546287, 'global_step': 116000}
117000
{'accuracy': 0.85623, 'loss': 0.5854276, 'global_step': 117000}
118000
{'accuracy': 0.8564297, 'loss': 0.59066033, 'global_step': 118000}
119000
{'accuracy': 0.85652953, 'loss': 0.5822454, 'global_step': 119000}
120000
{'accuracy': 0.85652953, 'loss': 0.5839621, 'global_step': 120000}
121000
{'accuracy': 0.85573083, 'loss': 0.5894563, 'global_step': 121000}
122000
{'accuracy': 0.8554313, 'loss': 0.58321273, 'global_step': 122000}
123000
{'accuracy': 0.85573083, 'loss': 0.5822993, 'global_step': 123000}
124000
{'accuracy': 0.8571286, 'loss': 0.5830142, 'global_step': 124000}
125000
{'accuracy': 0.8569289, 'loss': 0.5765861, 'global_step': 125000}
126000
{'accuracy': 0.8571286, 'loss': 0.58785826, 'global_step': 126000}
127000
{'accuracy': 0.85722846, 'loss': 0.5827239, 'global_step': 127000}
128000
{'accuracy': 0.8567292, 'loss': 0.

### Evaluate result

In [0]:
predictions = tpu_estimator.predict(input_fn=tpu_input_fn_valid)
pred_res = list(predictions)

In [0]:
id2sym = {value:key for key,value in sym2id.items() }

In [0]:
def tosym(ids):
  return [id2sym[id] for id in ids]

In [0]:
tosym(pred_res[0]['class_ids'][0][0:13])

['n', 'n', '{', '(', 'r', 'o', 'S', 'M', 'n', 'm', '<eol>', '<eol>', '<eol>']

In [0]:
tosym(sparse_label_dec[0][0:13])

['C', '_', 't', '=', 'C', '+', 'C', '=', '2', 'C', '<eol>', '<eol>', '<eol>']

In [0]:
tosym(pred_res[1]['class_ids'][0][0:18])

['\\frac',
 '{',
 '{',
 '2',
 '{',
 '2',
 '}',
 '}',
 '}',
 '{',
 'a',
 '_',
 '{',
 'n',
 '_',
 '(',
 '\\limits',
 '<eol>']

In [0]:
tosym(sparse_label_dec[1][0:18])

['\\frac',
 '{',
 '\\log',
 '_',
 '{',
 'b',
 '}',
 'x',
 '}',
 '{',
 '\\log',
 '_',
 '{',
 'b',
 '}',
 'a',
 '}',
 '<eol>']

In [0]:
tosym(pred_res[2]['class_ids'][0][0:13])

['\\frac', '{', '\\frac', '}', '{', '(', '1', 'y', '^', '{', '}', ')', '{']

In [0]:
tosym(sparse_label_dec[2][0:13])

['\\sqrt', '{', 'v', '^', '2', '-', '{', 'v', '_', 'v', '}', '^', '2']

Model seems overfitting.
Confirm by using the same input of training set(need to adjust tpu_model_fn too)

In [0]:
predictions = tpu_estimator.predict(input_fn=tpu_input_fn_predict_trainset)
pred_res2 = list(predictions)


INFO:tensorflow:Querying Tensorflow master (grpc://10.9.98.162:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 13729412150682042289)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3943709997331902754)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9486574995663304763)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 319607514878267032)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2382116045830419397)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/jo

In [0]:
tosym(pred_res2[0]['class_ids'][0][0:10])

['1', '_', '{', '=', '\\ldots', '_', '2', '<eol>', '<eol>', '<eol>']

In [0]:
tosym(decoder_sparse_target_data[0][0:10])

['f', '_', '2', ',', 'g', '_', '2', '<eol>', '<eol>', '<eol>']

In [0]:
tosym(pred_res2[1]['class_ids'][0][0:10])

['\\frac', '{', '1', '^', '2', '2', '}', '+', 'x', '^']

In [0]:
tosym(decoder_sparse_target_data[1][0:10])

['\\frac', '{', 'x', '^', '{', '2', '}', '+', 'y', '^']

In [0]:
tosym(pred_res2[2]['class_ids'][0][0:10])

['{', '.', '7', '<eol>', '<eol>', '<eol>', '<eol>', '<eol>', '<eol>', '<eol>']

In [0]:
tosym(decoder_sparse_target_data[2][0:10])

['7', '.', '1', '<eol>', '<eol>', '<eol>', '<eol>', '<eol>', '<eol>', '<eol>']

In [0]:
decoder_input_data[0:5]

array([[112,  77,  47,  65,   7,  51,  47,  65,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [112,  83,  57,  29,  75,  57,  65, 110,  34, 109,  75,  57,  65,
        110, 110,  57, 102,  75,  57,  65, 110, 110,  89,  83,  57,  49,
         75,  57,  65, 110, 110,  57,  48,  75,  57,  65, 110, 110,  34,
         90,  93,   8,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [0]:
tosym(sparse_label_dec[0][0:10])

['C', '_', 't', '=', 'C', '+', 'C', '=', '2', 'C']

In [0]:
tosym(pred_res[1]['class_ids'][0][0:13])

['f', '{', '2', 'x', '2', '2', '-', '}', '}', '{', '2', '_', '{']

In [0]:
tosym(sparse_label_dec[1][0:10])

['\\frac', '{', '\\log', '_', '{', 'b', '}', 'x', '}', '{']

In [0]:
tosym(pred_res[2]['class_ids'][0][0:13])

['a', 'u', '\\int', 't', '2', '2', '+', '2', '+', '2', 'a', '_', '3']

In [0]:
tosym(sparse_label_dec[2][0:10])

['s', 'q', 'r', 't', '{', 'v', '^', '2', '-', '{']

In [0]:
valid_raw_list[2]

('valid/TestEM2014GT_INKMLs/502_em_16.inkml',
 '\\sqrt{v^2-{v_v}^2}=\\frac {{v_v}^2}{\\sqrt{v^2-{v_v}^2}}',
 ['s',
  'q',
  'r',
  't',
  '{',
  'v',
  '^',
  '2',
  '-',
  '{',
  'v',
  '_',
  'v',
  '}',
  '^',
  '2',
  '}',
  '=',
  '\\frac',
  '{',
  '{',
  'v',
  '_',
  'v',
  '}',
  '^',
  '2',
  '}',
  '{',
  '\\sqrt',
  '{',
  'v',
  '^',
  '2',
  '-',
  '{',
  'v',
  '_',
  'v',
  '}',
  '^',
  '2',
  '}',
  '}'],
 ['260 181, 260 180, 262 177, 263 175, 264 174, 266 171, 268 170, 269 170, 271 170, 272 171, 273 173, 273 175, 274 178, 274 181, 274 184, 274 187, 276 192, 278 194, 280 194, 282 195, 287 193, 289 191, 291 188, 292 185, 293 182, 293 179, 293 176, 293 174, 292 172, 291 171, 290 170, 289 171, 288 170, 295 174, 297 174, 299 173, 300 172, 301 171, 300 172, 301 171',
  '321 137, 320 137, 320 136, 322 132, 323 132, 323 133, 323 137, 322 140, 321 142, 319 144, 317 146, 315 147, 313 148, 312 148, 312 147, 314 145, 316 145, 317 146, 319 146, 320 147, 322 148, 324 149, 326 150,

### TensorBoard

In [0]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      7 ?        00:00:02 node
     24 ?        00:00:03 jupyter-noteboo
    120 ?        00:00:00 tail
    128 ?        00:00:05 python3
    188 ?        00:00:00 python3
    220 ?        00:00:00 ps


In [0]:
!git clone https://github.com/mixuala/colab_utils

Cloning into 'colab_utils'...
remote: Enumerating objects: 243, done.
remote: Total 243 (delta 0), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (243/243), 65.93 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [0]:
kill_tensorboard()

In [0]:
import os
import colab_utils.tboard

ROOT = %pwd
colab_utils.tboard.launch_tensorboard(bin_dir=ROOT, log_dir=MODEL_DIR)
print(MODEL_DIR)

calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ...
calling unzip ngrok-stable-linux-amd64.zip ...


W0709 10:04:05.719727 140250003937152 deprecation_wrapper.py:119] From /content/colab_utils/tboard.py:115: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.



ngrok installed. path=/content/ngrok
status: tensorboard=False, ngrok=False
tensorboard url= https://6fa51247.ngrok.io
gs://karino2-tegashiki/sym_models/fe_layersbatchnorm


In [0]:
import re

def find_one_command(res_arr, word):
  return list(filter(lambda arr: arr[4] == word, res_arr))[0]

def kill_tensorboard():
  ps_res = !ps
  res_arr = [re.split(r' +', one) for one in ps_res[1:]]
  # pid_ngrok = find_one_command(res_arr, "ngrok")[1]
  pid_tb = find_one_command(res_arr, "tensorboard")[1]
  # !kill {pid_ngrok}
  !kill {pid_tb}

In [0]:
# kill_tensorboard()

In [0]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      6 ?        00:00:04 node
     23 ?        00:00:05 jupyter-noteboo
    124 ?        00:00:00 tail
   1568 ?        00:00:04 python3
   1622 ?        00:00:00 python3
   1634 ?        00:00:00 ps


In [0]:
ps_res = !ps

In [0]:
res_arr = [re.split(r' +', one) for one in ps_res[1:]]

In [0]:
find_one_command(res_arr, "ngrok")

['', '2836', '?', '00:00:10', 'ngrok']

In [0]:
find_one_command(res_arr, "tensorboard")

['', '2833', '?', '00:01:04', 'tensorboard']

In [0]:
pid_ngrok = find_one_command(res_arr, "ngrok")[1]
pid_ngrok

'2836'

In [0]:
!kill 2833

In [0]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      7 ?        00:00:22 node
     24 ?        00:00:51 jupyter-noteboo
    116 ?        00:00:00 tail
    124 ?        00:26:11 python3
    167 ?        00:00:00 python3
   2833 ?        00:01:19 tensorboard
   3718 ?        00:00:00 ps


In [0]:
list(filter(lambda arr: arr[4] == "ngrok", res_arr))

[['', '2836', '?', '00:00:10', 'ngrok']]

In [0]:
re.split(r' +', ps_res[1])

['', '1', '?', '00:00:00', 'run.sh']

In [0]:
!kill 10785
!kill 10788

In [0]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      6 ?        00:00:19 node
     21 ?        00:00:46 jupyter-noteboo
    111 ?        00:00:01 tail
    120 ?        00:00:56 python3
    400 ?        00:00:00 python3
   7345 ?        00:00:47 python3
   7406 ?        00:00:00 python3
   8279 ?        00:00:00 ps


# Try and error

### TCN n estimate

In [0]:
import math

In [0]:
1+499/14

36.642857142857146

In [0]:
# 1+2*(k-1)*(2^n-1) #> 500
# 1+2*7*(2^n-1) > 500
# 2^n-1 > 499/14
# 2^n > 1+499/14
math.log2(1+ (499/14.0))

5.19546009354945

I set n = 6

In [0]:
# for target == 2000 case
math.log2(1+ (1999/14.0))

7.1677765348637354

In [0]:
# for target == 300 case
math.log2(1+ (300/14.0))

4.487265826834022

In [0]:
2**0

1